In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [4]:
#year=[(('https://www.imdb.com/search/title?year=%s&title_type=feature&')%(y)) for y in range(2012,2018)]

In [15]:
def get_ratings_box(year):
    
    url='https://www.imdb.com/search/title?title_type=feature&year=%s-01-01,%s-12-31&sort=boxoffice_gross_us,desc'%(year,year)
    list_names=[]
    list_ratings=[]
    list_gross=[]
    list_votes=[]
    list_genre=[]
    list_certificate=[]
    list_length=[]
    list_year=[year]
        
    gross=0
    
    while gross!='$0.00M':
        response = requests.get(url)
        results_page = BeautifulSoup(response.content,'lxml')
    
        next_page=results_page.find('div',attrs={"class":"desc"})
        links=next_page.find_all('a')
        for i in links:
            link=i.get('href')
            if link.endswith('t'):
                next_page_link=link 
                url='https://www.imdb.com'+ next_page_link
                #test
                #print(next_page_link)
           
        rawnames=results_page.find_all('div',attrs={"class":"lister-item-content"}) 
        for row in rawnames:
            names=row.find_all('h3',attrs={"class":"lister-item-header"}) 
            for name in names:
                for x in name.find_all('a'):
                    list_names.append(x.text)
                for y in row.find_all('span',attrs={"class":"genre"}):
                    list_genre.append(y.text.strip())
                #for z in row.find_all('span',attrs={"class":"runtime"}):
                    #list_length.append(z.text.strip())
                #for w in row.find_all('span',attrs={"class":"certificate"}):
                    #list_certificate.append(w.text.strip())
                    
        rawratings=results_page.find_all('div',attrs={"class":"inline-block ratings-imdb-rating"})
        for row in rawratings:
            ratings=row.get('data-value')
            list_ratings.append(ratings)
                        
        rawdatavalues=results_page.find_all('div',attrs={"class":"lister-item-content"}) 
        for row in rawdatavalues:
            datavalues=row.find_all('p',attrs={"class":"sort-num_votes-visible"}) 
            for datavalue in datavalues:
                for y in datavalue.find_all('span',attrs={"name":"nv"}):
                    if y.text.startswith('$'):
                        gross=y.text
                        grossdatavalue=y.get('data-value')
                        list_gross.append(grossdatavalue)
                        #test
                        #print(gross)
                    else:
                        votes=y.text
                        votesdatavalue=y.get('data-value')
                        list_votes.append(votesdatavalue)
                    
        full_list=list(zip(list_names,list_genre,list_ratings,list_votes,list_gross,[year for i in range(0,len(list_names))]))
        
    return full_list

In [4]:
import pandas as pd

In [23]:
list_=[]
for i in range(2012,2018):
    year=i
    list_.append(get_ratings_box(i))

In [24]:
names=[]
genre=[]
rating=[]
votes=[]
boxoffice=[]
year=[]
    
for i in list_:
    for s in i:
        names.append(s[0])
        genre.append(s[1])
        rating.append(s[2])
        votes.append(s[3])
        boxoffice.append(s[4])
        year.append(s[5])

dict_={'Name':names,'Genre':genre,'Rating':rating,'Votes':votes,'BoxOffice':boxoffice,'Year':year}
df=pd.DataFrame(dict_)
df = df.reset_index(drop=True)

In [23]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('IMDB.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='movie_info')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

workbook  = writer.book
worksheet = writer.sheets['movie_info']